In [1]:
from __future__ import division, print_function
from six import print_ as print
import idi.simulation as sim
import numpy as np
from numpy import pi
!hostname

psanagpu106


In [2]:
import os
os.environ['PATH']+=':/usr/local/cuda/bin/' # cuda must be in path

**Settings**

In [3]:
outfile = '/reg/data/ana15/xpp/xppx37817/scratch/zimmf/lammps.npz'
Ndet = 1024
detz = 4.1  # in cm
pixelsize = 75 # in um
Nimg = 200
E = 8040  # in ev
rndphase = True
cuda = True
simtype = 'lammps'
import glob
inputfiles=glob.glob("atompos*.npz")

atomname='Cu'

In [4]:
#check inputfiles
files=[]
for f in inputfiles:
    try:
        t=np.load(f)
    except IOError:
        print("Could not read %s, skipping."%f)
    else:
        files.append(f)
inputfiles = files
print("Found %i inputfiles"%len(inputfiles))

Could not read atompos_timestep1000000.npz, skipping.
Found 44 inputfiles


**Simulation**

In [5]:
print("preparing")
_detz = detz * 1e4 # in um
k = 2 * pi / (1.24 / E)  # in 1/um

from idi.simulation.simobj import atoms
class lammps(atoms):
    import numpy as np
    def __init__(self,files,atomname,E):
        self.files=files
        self.currentfile=0
        self.atomname=atomname
        f=np.load(files[0])
        bounds=f['bounds']
        pos=(f[self.atomname]*(bounds[:,1]-bounds[:,0])+bounds[:,0])*1e-3 #in um
        atoms.__init__(self, E, pos)

    def get(self, rndPhase=True):
        f=np.load(files[self.currentfile])
        bounds=f['bounds']
        self.currentfile=(self.currentfile+1)%len(self.files)       
        self._pos=(f[self.atomname]*(bounds[:,1]-bounds[:,0])+bounds[:,0])*1e-3 #in um
        return atoms.get(self)

simobject = lammps(inputfiles, atomname, E)
simobject.rndPhase = rndphase


preparing


In [ ]:
print("simulating")
if cuda:
    result = sim.cuda.simulate(Nimg, simobject, Ndet, pixelsize, _detz, k)
else:
    print("using cpu")
    result = sim.cpu.simulate(Nimg, simobject, Ndet, pixelsize, _detz, k)
result = np.abs(result * result.conjugate())

simulating
0.. 1.. 2.. 3.. 4.. 5.. 6.. 7.. 8.. 9.. 10.. 11.. 12.. 13.. 14.. 15.. 16.. 17.. 18.. 19.. 20.. 21.. 22.. 23.. 24.. 25.. 26.. 27.. 28.. 29.. 30.. 31.. 32.. 33.. 34.. 35.. 36.. 37.. 38.. 39.. 40.. 41.. 42.. 43.. 44.. 45.. 46.. 47.. 48.. 49.. 50.. 51.. 52.. 53.. 54.. 55.. 56.. 57.. 58.. 59.. 60.. 61.. 62.. 63.. 64.. 65.. 66.. 67.. 68.. 69.. 70.. 71.. 72.. 73.. 74.. 75.. 76.. 77.. 78.. 79.. 80.. 81.. 82.. 83.. 84.. 85.. 86.. 87.. 88.. 89.. 90.. 91.. 92.. 93.. 94.. 95.. 96.. 97.. 98.. 99.. 100.. 101.. 102.. 103.. 104.. 105.. 106.. 107.. 108.. 109.. 110.. 111.. 112.. 113.. 114.. 115.. 116.. 117.. 118.. 119.. 120.. 121.. 122.. 123.. 124.. 125.. 126.. 127.. 128.. 129.. 130.. 131.. 132.. 133.. 134.. 135.. 136.. 137.. 138.. 139.. 140.. 141.. 142.. 143.. 144.. 145.. 146.. 147.. 148.. 149.. 150.. 151.. 152.. 153.. 154.. 155.. 156.. 157.. 158.. 159.. 160.. 161.. 162.. 163.. 164.. 165.. 166.. 167.. 168.. 169.. 170.. 171.. 172.. 173.. 174.. 175.. 176.. 177.. 178.. 179.. 180.. 181.. 182.. 1

In [ ]:
print("saving")
np.savez_compressed(outfile, result = result, settings = ({
    'Natoms': simobject.get().shape[0],
    'Ndet': Ndet,
    'detz': detz,
    'Nimg': Nimg,
    'atomname': atomname,
    'inputfiles': inputfiles,
    'pixelsize': pixelsize,
    'E': E,
    'rndphase': rndphase,
    }, [simtype]))

In [ ]:
print("done")

**Show Simulation object**

In [ ]:
pos=simobject.get()

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(pos[:,0]*1e4,pos[:,1]*1e4,pos[:,2]*1e4)
